In [ ]:
!pip install ktrain

In [ ]:
import ktrain
from ktrain import text
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
MODEL_NAME = 'albert-base-v2'
t = text.Transformer(MODEL_NAME, maxlen= 120, classes= ['sexual_violence', 'Physical_violence', 'emotional_violence',
       'Harmful_Traditional_practice', 'economic_violence'])

In [ ]:
train_df = pd.read_csv("/content/Train.csv")
random_seed = 12342
x_train, x_val, y_train, y_val = train_test_split(train_df['tweet'], 
train_df['type'], shuffle=True, test_size = 0.2, random_state=random_seed, stratify=train_df['type'])

In [ ]:

import re
def text_preprocessing(text):
    """
    - Remove entity mentions (eg. '@united')
    - Correct errors (eg. '&amp;' to '&')
    @param    text (str): a string to be processed.
    @return   text (Str): the processed string.
    """
    # Remove '@name'
    text = re.sub(r'(@.*?)[\s]', ' ', text)

    # Replace '&amp;' with '&'
    text = re.sub(r'&amp;', '&', text)

    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [ ]:
train_df.tweet.apply(text_preprocessing)

In [ ]:
trn = t.preprocess_train(numpy.array(x_train), numpy.array(y_train))
val = t.preprocess_test(numpy.array(x_val), numpy.array(y_val))


In [ ]:
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data= trn, val_data= val, batch_size=6)

In [ ]:
learner.autofit(1e-5)

In [ ]:
predictor = ktrain.get_predictor(learner.model, t)

In [ ]:
import numpy

In [ ]:
(x_train_bert, y_train_bert), (x_val_bert, y_val_bert), prepoc = text.texts_from_array(x_train=numpy.array(x_train), y_train=numpy.array(y_train), x_test= numpy.array(x_val), y_test=numpy.array(y_val),
                                          class_names=['sexual_violence', 'Physical_violence', 'emotional_violence',
       'Harmful_Traditional_practice', 'economic_violence'],
                                          preprocess_mode='bert',
                                          lang= 'en',
                                          maxlen=200,
                                          max_features= 35000)

In [ ]:
model = text.text_classifier('bert', train_data=(x_train_bert, y_train_bert), preproc=prepoc)
learner = ktrain.get_learner(model, train_data=(x_train_bert, y_train_bert), val_data=(x_val_bert, y_val_bert), batch_size=16)

In [ ]:
learner.autofit(1e-5)

In [ ]:
predictor = ktrain.get_predictor(learner.model, prepoc)



In [ ]:
test_df = pd.read_csv("/content/Test.csv")

In [ ]:
sample_submission = pd.read_csv("/content/SampleSubmission.csv")

In [ ]:
sample_submission["type"] = predictor.predict(test_df.tweet.to_list())

In [ ]:
sample_submission

In [ ]:
sample_submission.to_csv('bert.csv', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
sample_submission.to_csv('/content/drive/MyDrive/Data_sendy/bert.csv')

1